Prepare

In [1]:
data_dir = 'data/state'

In [2]:
import pathlib
def image_count(data_dir):
    data_dir = pathlib.Path(data_dir)
    image_count = len(list(data_dir.glob('*/*.jpg')))

    return image_count

In [3]:
image_count = image_count(data_dir)
print(image_count)

35887


In [4]:
import tensorflow as tf
def split_data(data_dir, split_size=0.2, seed = 123, color_mode='grayscale'):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=split_size,
        color_mode=color_mode,
        seed=seed,
        subset="training",
        image_size=(48,48),
        batch_size=64)
    
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=split_size,
        color_mode=color_mode,
        seed=seed,
        subset="validation",
        image_size=(48,48),
        batch_size=64)

    return (train_ds, val_ds)

2022-12-26 16:03:49.071142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-26 16:03:50.613927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tvc/gtrbtm_ws/devel/lib:/home/tvc/catkin_ws/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu:/usr/local/cuda-11.7/lib64
2022-12-26 16:03:50.614024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_

In [5]:
(train_ds, val_ds) = split_data(data_dir, split_size=0.2)

Found 35887 files belonging to 7 classes.
Using 28710 files for training.


2022-12-26 16:03:52.891544: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-26 16:03:52.952578: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-26 16:03:52.952893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-26 16:03:52.954132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Found 35887 files belonging to 7 classes.
Using 7177 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


The RGB channel value is in the range [0, 255] . This is not ideal for neural networks; in general you should find a way to reduce your input value.

In [7]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)

In [8]:
# train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [9]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 48, 48, 1)
(64,)


Configure the datasets according to performance

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

Create network

In [12]:
num_classes = 7

emotion_model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    # tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(num_classes, activation='softmax')]
)

In [13]:
emotion_model.build((None, 48,48,1))
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 48, 48, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                        

Compile and train

In [14]:
from keras.optimizers import Adam

In [15]:
# emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model.compile(
  optimizer=Adam(learning_rate=0.0001, decay=1e-6),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [16]:
emotion_model_info = emotion_model.fit(train_ds, validation_data=val_ds, batch_size = 64, epochs = 50)

Epoch 1/50


2022-12-26 16:03:56.371563: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-12-26 16:03:57.120050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2022-12-26 16:03:57.814454: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


449/449 [==============================] - 15s 28ms/step - loss: 1.7278 - accuracy: 0.3091 - val_loss: 1.5954 - val_accuracy: 0.3867
Epoch 2/50
449/449 [==============================] - 12s 27ms/step - loss: 1.5382 - accuracy: 0.4096 - val_loss: 1.5098 - val_accuracy: 0.4186
Epoch 3/50
449/449 [==============================] - 12s 27ms/step - loss: 1.4304 - accuracy: 0.4557 - val_loss: 1.4465 - val_accuracy: 0.4467
Epoch 4/50
449/449 [==============================] - 12s 27ms/step - loss: 1.3482 - accuracy: 0.4910 - val_loss: 1.3873 - val_accuracy: 0.4758
Epoch 5/50
449/449 [==============================] - 11s 25ms/step - loss: 1.2805 - accuracy: 0.5159 - val_loss: 1.3440 - val_accuracy: 0.4994
Epoch 6/50
449/449 [==============================] - 11s 25ms/step - loss: 1.2232 - accuracy: 0.5389 - val_loss: 1.2953 - val_accuracy: 0.5141
Epoch 7/50
449/449 [==============================] - 11s 25ms/step - loss: 1.1706 - accuracy: 0.5639 - val_loss: 1.2626 - val_accuracy: 0.5268
Epo

Save the model weights

In [17]:
emotion_model.save_weights('models/model.h5')